# Start Here: Extend happens below

In [12]:
import os

non_fname = "c:/Python27/qp2/sample_both/non_native/p452p453-part2_ch1_pitches.txt"
native_fname = "c:/Python27/qp2/sample_both/native/p226p227-part2_ch2_pitchs.txt"

non_infile = open(non_fname, "r")
non_data = non_infile.readlines()

native_infile = open(native_fname, "r")
native_data = native_infile.readlines()

labels = ["NonNative", "Native"]
non_infile.close()
native_infile.close()


all_data = [non_data, native_data]
#Only do this ONCE!  
#del data[0]

In [2]:
## Run this EXACTLY ONCE
for item in all_data:
    del item[0]

for item in all_data:
    print item[0]

silent	0	23.86	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA

silent	0	7.42179138322	NA	NA	NA	NA	NA	NA	NA	NA	NA	238.2	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	119.3	148.2	NA	NA	NA	NA	NA	NA	NA



In [8]:
print len(all_data)
    

2


In [49]:
print data[3]
import math
print math.log(116.5)


STOP	24.56	24.88	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	NA	113.0	114.0	113.1	112.8	113.5	117.4	125.4	NA	NA	NA	NA	NA	NA	NA	NA

4.75789127301


In [4]:
## Extended for long lists of data (2/16/2016)
##Takes as input praat f0 extraction file, returns a list of lists [ipu, x...n of F0 values]
## takes as input data that's been extracted from 1 praat tier
## NOTE:  For a tier other than IPUs, must modify the PRAAT SCRIPT.

ipu_out = []
intermediate = []
for data in all_data:
    for element in data:
        ipu = element.split('\t')
        if ipu[0]=="silent" or ipu[0]=="#":
            pass
        else:
            txt = ipu[0]
            f0s = ipu[1:]
            f0_list = [txt]
            for i in range(len(f0s)):
                try:
                    f0 = float(f0s[i])
                    f0_list.append(f0)
                except ValueError:
                    pass
            intermediate.append(f0_list)
        f0_list = []
    ipu_out.append(intermediate)
    intermediate = []


print len(ipu_out)
for j in range(len(ipu_out)):
    for i in range(20):
        print ipu_out[j][i]


2
['OH', 23.86, 23.99, 124.8, 122.3, 127.4, 125.6]
['STOP', 24.56, 24.88, 113.0, 114.0, 113.1, 112.8, 113.5, 117.4, 125.4]
['OH I HAVE A QUESTION', 26.27, 27.14, 189.3, 175.9, 177.8, 186.6, 189.5, 184.0, 182.6, 182.9, 178.0, 157.1, 152.1, 162.2, 166.5]
['WHAT IS A CONFIDENCE', 27.53, 28.86, 133.0, 114.7, 128.4, 143.3, 160.9, 179.9, 171.2, 152.6, 133.7, 116.1, 155.4, 156.2]
['A NUMBER SO', 29.2, 30.17, 117.1, 115.5, 118.4, 123.0, 128.0, 134.9, 135.6, 124.5, 114.3, 109.4, 148.0, 142.8, 135.0]
['YEAH', 36.06, 36.29, 133.5, 127.8, 114.7, 106.5, 106.6, 104.7, 101.8, 108.7, 110.9, 109.9, 119.1]
['OH SO', 45.27, 45.71, 120.8, 120.3, 118.7, 118.4, 115.7, 112.3, 148.8, 142.3, 139.1, 136.8, 135.2, 130.7, 127.5]
["IT'S", 45.9, 46.26, 137.8, 138.9, 138.1, 136.0, 131.9, 128.4]
["IT'S A NUMBER FOR", 46.59, 47.48, 142.7, 136.6, 134.0, 134.1, 127.3, 125.3, 124.9, 121.3, 120.2, 121.8, 109.9, 112.1, 112.0, 116.2, 113.2, 112.2, 108.9]
['HOW MUCH I', 47.68, 48.54, 130.3, 125.5, 124.6, 122.3, 125.5, 127.2,

In [44]:
## some shit code about logs and list comprehensions 
#import math
#logs = ipu_out[:]
#for item in logs:
#    f0s = item[1:]
#    loggys = [math.log(item)for item in f0s]

In [6]:
###NOTES ABOUT THIS FUNCTION: 
## Extended for large data lists (2/16/2016)
## log normalizes each F0 point in the IPU, then calculates min, max, and std of the normalized F0s
## also calculates mean and the z-normalized mean of the logF0 points per IPU
## currently (2/15/2016) returns a list of (ipuString, [seconds, minF0, maxF0, std, meanF0, znorm_meanF0, whisper])
## Goal is to output format that's good for sklearn
## NOTE:  For whispered ipus, all values are "NaN"
## NOTE: For one F0 point ipus, min, max, and mean F0 features are log of the F0 point, std&znorm are NaN, whisper = 0


## QUESTIONS FOR ANDREW (2/3/2016):
### z-normalized is returning between 1 and -1, is that okay?
## Is the normalization I'm doing okay
## can I z-score normalize already log normalized f0 means?
## QUESTIONS 2/15/2016
## is my zscore normalization correct?
## handling whisperes and f0 points okay?


import scipy
import math
from scipy import stats
logs_long = ipu_out[:] #make a copy cause I'm fucking with the list

feature_set = []
intermediate = []


for logs in logs_long: 
    for item in logs:
    ## assign text, calculate length    
        txt = item[0]
        start_time = item[1]
        end_time = item[2]
        time = end_time-start_time

        f0s = item[3:]
        if bool(f0s) == False:
        ### for whispered ipus (with no F0)
            minF0 = "NaN"
            maxF0 = "NaN"
            std = "NaN"
            meanF0 = "NaN"
            znorm_mean = "NaN"
            whisper = 1

        elif len(f0s) ==1:
        ### for ipus with only one F0 point
            oneVal = math.log(f0s[0])
            minF0 = oneVal
            maxF0 = oneVal
            std = "NaN"
            meanF0 = oneVal
            znorm_mean = "NaN"
            whipser = 0

        else:
        ### for valid ipus with many F0 points
            logF0s = [math.log(item) for item in f0s]
            minF0 = min(logF0s)
            maxF0 = max(logF0s)
            meanF0 = sum(logF0s)/len(logF0s)
            std = scipy.stats.tstd(logF0s)
            for point in logF0s:
                znorm_mean = (point-meanF0)/std
            whisper = 0
        #print "min:%f, max:%f, std:%f, mean:%f, znormMean:%f" %(minF0, maxF0, std, meanF0, znorm_mean)

        features = [time, minF0, maxF0, std, meanF0, znorm_mean, whisper]
        set_tup = (txt, features)
        intermediate.append(set_tup)
    feature_set.append(intermediate)
    intermediate = []

for i in range(len(feature_set)):
    print len(feature_set[i])

432
513


##### testing syllable code stuff  - DO NOT RUN####
import pronouncing
text = "this is a sentence"
phones = [pronouncing.phones_for_word(p)[0] for p in text.split()]
print phones
syl = sum([pronouncing.syllable_count(p) for p in phones])
print syl

pronouncing.phones_for_word("and")

In [7]:
####Calculates syllables for each IPU
## NOTE: Assumes words not in the dictionary are all monosyllabic (assumption based on um, mm, and false starts)
### But this is definitely not accurate, there will be names that are wrong too.
## Extended for large data (2/16/2016)

import pronouncing
syl_list = []
intermediate = []

def getWeirdSyllables(text):
    """Gets syllables count for IPUs that contain OOVs.  NOTE:  OOVs always return monosyllabic"""
    syl_sum = 0
    checkit = text.split()
    for word in checkit:
        valid = pronouncing.phones_for_word(word)
        #print valid
        if valid:
            syl = sum([pronouncing.syllable_count(p)for p in valid[0]])
        else:
            syl = 1
        syl_sum = syl_sum+syl
    return syl_sum

    

for features in feature_set:    
    for item in features:
        text = item[0]
        text = text.lower()
        try:
            phones = [pronouncing.phones_for_word(p)[0] for p in text.split()]
            syl = sum([pronouncing.syllable_count(p) for p in phones])
        except IndexError:
            syl = getWeirdSyllables(text)
        intermediate.append(syl)
    syl_list.append(intermediate)
    intermediate = []



print len(syl_list)
for i in range(len(syl_list)):
    print len(syl_list[i])

    



2
432
513


In [9]:
## replaces time with seconds per syllable
## feature_set is now done for a baseline, for many files (2/16/2016)

num_datapoint = len(feature_set)

for num in range(num_datapoint):
    for i in range(len(feature_set[num])):
        length = feature_set[num][i][1][0]
        ##feature set [each featurelist][each ipu][list of features][seconds]
        syl_num = syl_list[num][i] #get the # of syllables

        syl_per_sec = syl_num / length
        feature_set[num][i][1][0]= syl_per_sec #replace with syllables per second


## Build in quick check
for num in range(num_datapoint):
    for i in range(10):
        print feature_set[num][i]



('OH', [7.692307692307751, 4.8064770426931265, 4.847331743138063, 0.016970423440826141, 4.828405873950154, 0.27673912205665518, 0])
('STOP', [3.1249999999999973, 4.725616339063959, 4.83150862819882, 0.038696063092143122, 4.749481908829592, 2.1197691138218886, 0])
('OH I HAVE A QUESTION', [6.8965517241379235, 5.024538199265247, 5.244389024522481, 0.072447648401045478, 5.166574413312438, -0.71195000845873457, 0])
('WHAT IS A CONFIDENCE', [4.511278195488727, 4.742320024135325, 5.192401140956488, 0.14482053019350744, 4.970315558675897, 0.55808163816066525, 0])
('A NUMBER SO', [4.123711340206175, 4.695010889987881, 4.997212273764115, 0.09291178368111809, 4.837453889217608, 0.7299492759022822, 0])
('YEAH', [4.347826086956581, 4.623010104116422, 4.894101477840304, 0.084923946652542492, 4.724998253549774, 0.64722878502531378, 0])
('OH SO', [4.545454545454569, 4.721173861744398, 5.002603122398992, 0.088934762562913314, 4.849922869918544, -0.020312701895217805, 0])
("IT'S", [2.777777777777782, 4

In [13]:
print len(feature_set)
print len(labels)

2
2
